# 埋め込み表現

## セットアップ

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls

In [ ]:
%cd '/content/drive/MyDrive/研究'

In [ ]:
# ライブラリのインポート
import os
import glob
import numpy as np
import pandas as pd
import torch

!pip install torch transformers
from transformers import AutoTokenizer, AutoModel

## モデル

In [ ]:
# ModernBERT
model_name = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model     = AutoModel.from_pretrained(model_name)
device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# カテゴリファイル
cat_df = pd.read_csv("category.csv", dtype={"TransissonId": str})
cond = ''

In [ ]:
model.to(device)
model.eval()

# 各ファイルごとに処理
results = {}  # 日付 → 平均ベクトル

for fp in glob.glob(os.path.join("kyodo", "*.csv")):
    # ファイル名から日付を取得（例: "2023-12-24.csv" → "2023-12-24"）
    date_str = os.path.splitext(os.path.basename(fp))[0]

    # 元データ読み込み
    df = pd.read_csv(fp, dtype={"TransissonId": str})

    # カテゴリマスタとマージ＆フィルタ
    merged = df.merge(cat_df, on="TransissonId", how="inner")
    filtered = merged[merged["category"] == cond]

    # テキスト結合＆埋め込み取得
    embeddings = []
    for _, row in filtered.iterrows():
        text = " ".join([
            row.get("Headline", "") or "",
            row.get("SubHeadline", "") or "",
            row.get("NewsContent", "") or ""
        ])
        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=8192
        ).to(device)
        with torch.no_grad():
            out = model(**inputs)
        # CLSトークンの埋め込みを numpy ベクトルに
        vec = out.last_hidden_state[:,0,:].squeeze().cpu().numpy()
        embeddings.append(vec)

    # embedding の平均を計算（記事がない日はゼロベクトル）
    if embeddings:
        embedding = np.stack(embeddings, axis=0).mean(axis=0)
    else:
        embedding = np.zeros(model.config.hidden_size, dtype=float)

    results[date_str] = embedding

# 日付インデックスのdataframeにまとめる
emb_df = pd.DataFrame.from_dict(results, orient="index")
emb_df.index = pd.to_datetime(emb_df.index)
emb_df.sort_index(inplace=True)
emb_df.columns = [f"dim_{i}" for i in range(emb_df.shape[1])]

# 結果表示
print(emb_df)

In [ ]:
# 必要に応じて CSV 出力なども可能
# emb_df.to_csv("daily_average_embeddings.csv")